# [AlexNet]

*KU LeeDongGyu*

### Install Packages

In [0]:
pip install opencv-python

### Module

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
cd /content/drive/My Drive/Colab Notebooks/project

/content/drive/My Drive/Colab Notebooks/project


In [0]:
import sys

In [5]:
# 모듈로 받을 경로 확인
sys.path

['',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython']

In [0]:
# 내 노트북이 아닌, 전산실 컴퓨터의 colab에서 돌렸으므로, 다시돌리려면 경로 수정할것!
sys.path.append("/content/drive/My Drive/Colab Notebooks/project")

In [7]:
import pandas as pd
import numpy as np
import cv2
import os
from lrn import LRN #만든 모듈, class
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K

In [8]:
os.getcwd()

'/content/drive/My Drive/Colab Notebooks/project'

### F1-score Function

In [0]:
'''
아래 코드는 f1-score를 정의한 함수.
tensorflow2.0 버전 이후로 f1-score 지원이 metrics애서 사라졌다.
'''

def recall(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_target_yn = K.round(K.clip(y_target, 0, 1))  # 실제값을 0(Negative) 또는 1(Positive)로 설정한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1))  # 예측값을 0(Negative) 또는 1(Positive)로 설정한다
    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn)
    # (True Positive + False Negative) = 실제 값이 1(Positive) 전체
    count_true_positive_false_negative = K.sum(y_target_yn)
    # Recall =  (True Positive) / (True Positive + False Negative)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())
    # return a single tensor value
    return recall

def precision(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1))  # 예측값을 0(Negative) 또는 1(Positive)로 설정한다
    y_target_yn = K.round(K.clip(y_target, 0, 1))  # 실제값을 0(Negative) 또는 1(Positive)로 설정한다
    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn)
    # (True Positive + False Positive) = 예측 값이 1(Positive) 전체
    count_true_positive_false_positive = K.sum(y_pred_yn)
    # Precision = (True Positive) / (True Positive + False Positive)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    precision = count_true_positive / (count_true_positive_false_positive + K.epsilon())
    # return a single tensor value
    return precision

def f1score(y_target, y_pred):
    _recall = recall(y_target, y_pred)
    _precision = precision(y_target, y_pred)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    _f1score = (2 * _recall * _precision) / (_recall + _precision + K.epsilon())
    # return a single tensor value
    return _f1score

## 1. Data Handling
---

In [0]:
# data import
x_train = pd.read_csv("mydata/X_train.csv",header=0,index_col=0)
x_valid = pd.read_csv("mydata/X_private_test.csv",header=0,index_col=0)
x_test = pd.read_csv("mydata/X_public_test.csv",header=0,index_col=0)
y_train = pd.read_csv("mydata/y_train.csv",header=0,index_col=0)
y_valid = pd.read_csv("mydata/y_private_test.csv",header=0,index_col=0)
y_test = pd.read_csv("mydata/y_public_test.csv",header=0,index_col=0)


y_train = np.array(y_train).reshape([-1,])
y_valid = np.array(y_valid).reshape([-1,])
y_test = np.array(y_test).reshape([-1,])

In [0]:
# data handling
# uint는 부호없는 정수로, 타입을 바꿔줘야함!
size = 227
x_train = np.array(x_train).reshape([-1,48,48,1])

In [0]:
x_train.shape

(28698, 48, 48, 1)

In [0]:
x_train_zoom = np.zeros([x_train.shape[0],size,size,1],dtype="float32")

for i in range(x_train.shape[0]):
    x_train_zoom[i,:] = cv2.resize(x_train[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,1) /255

In [0]:
x_train_zoom.shape

(28698, 227, 227, 1)

In [0]:
x_valid = np.array(x_valid).reshape([-1,48,48,1])
x_valid_zoom = np.zeros([x_valid.shape[0],size,size,1],dtype="float32")
for i in range(x_valid.shape[0]):
    x_valid_zoom[i,:] = cv2.resize(x_valid[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,1) /255

In [0]:
x_valid_zoom.shape

(3589, 227, 227, 1)

In [0]:
x_test = np.array(x_test).reshape([-1,48,48,1])
x_test_zoom = np.zeros([x_test.shape[0],size,size,1],dtype="float32")
for i in range(x_test.shape[0]):
    x_test_zoom[i,:] = cv2.resize(x_test[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,1) /255

In [0]:
x_test_zoom.shape

(3588, 227, 227, 1)

## 2. Alexnet
---

### 1) Almost Original Alexnet

In [0]:
# Alexnet을 최대한 논문에 가깝게 맞춰 모형작성.
# 또한, Data Augmentation은 컴퓨터 성능의 한계로 하지 않음.
# 출처 : https://github.com/eweill/keras-deepcv/blob/master/models/classification/alexnet.py

def Alexnet(img_shape=(227, 227, 1), n_classes=10, l2_reg=0.,weights=None):

    # Initialize model
    alexnet = Sequential()

    # Layer 1
    alexnet.add(Conv2D(96, (11, 11), input_shape=img_shape, strides=4,kernel_regularizer=l2(l2_reg)))
    alexnet.add(Activation('relu'))
    alexnet.add(LRN(name='layer1_LRN'))
    alexnet.add(MaxPooling2D(pool_size=(3, 3),strides=2))

    # Layer 2
    alexnet.add(Conv2D(256, (5, 5), padding='same',strides=1,kernel_regularizer=l2(l2_reg)))
    alexnet.add(Activation('relu'))
    alexnet.add(LRN(name='layer2_LRN'))
    alexnet.add(MaxPooling2D(pool_size=(3, 3),strides=2))
    
    # Layer 3
    alexnet.add(Conv2D(384, (3, 3), padding='same', strides=1,kernel_regularizer=l2(l2_reg)))
    alexnet.add(Activation('relu'))
    alexnet.add(LRN(name='layer3_LRN'))
    
    # Layer 4
    alexnet.add(Conv2D(384, (3, 3), padding='same'))
    alexnet.add(Activation('relu'))
    alexnet.add(LRN(name='layer4_LRN'))

    # Layer 5
    alexnet.add(Conv2D(256, (3, 3), padding='same'))
    alexnet.add(Activation('relu'))
    alexnet.add(LRN(name='layer5_LRN'))
    alexnet.add(MaxPooling2D(pool_size=(3, 3),strides=2))

    # Layer 6
    alexnet.add(Flatten())
    alexnet.add(Dense(4096,kernel_regularizer=l2(l2_reg)))
    alexnet.add(Activation('relu'))
    alexnet.add(Dropout(0.5))

    # Layer 7
    alexnet.add(Dense(4096,kernel_regularizer=l2(l2_reg)))
    alexnet.add(Activation('relu'))
    alexnet.add(Dropout(0.5))

    # Layer 8
    alexnet.add(Dense(n_classes))
    alexnet.add(Activation('softmax'))

    if weights is not None:
        alexnet.load_weights(weights)

    return alexnet

In [0]:
model = Alexnet(img_shape=(227, 227, 1), n_classes=7, l2_reg=0.,weights=None)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
#모수가 데이터에 비해 굉장히 많지만, 일단 진행
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 55, 55, 96)        11712     
_________________________________________________________________
activation (Activation)      (None, 55, 55, 96)        0         
_________________________________________________________________
layer1_LRN (LRN)             (None, 55, 55, 96)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
activation_1 (Activation)    (None, 27, 27, 256)       0         
_________________________________________________________________
layer2_LRN (LRN)             (None, 27, 27, 256)       0

In [0]:
# 여기는 학습의 효율을 위해. Adam으로 넘어간다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='sparse_categorical_crossentropy',
              metrics=['accuracy',f1score])

In [0]:
model.fit(x_train_zoom,y_train,batch_size=128, validation_data=(x_valid_zoom,y_valid) , epochs=10)

Train on 28698 samples, validate on 3589 samples
Epoch 1/10
28698/28698 [==============================] - 3031s 106ms/sample - loss: 1.8141 - acc: 0.2514 - f1score: 0.0000e+00 - val_loss: 1.8168 - val_acc: 0.2449 - val_f1score: 0.0000e+00
Epoch 2/10
28698/28698 [==============================] - 3042s 106ms/sample - loss: 1.8126 - acc: 0.2514 - f1score: 0.0000e+00 - val_loss: 1.8181 - val_acc: 0.2449 - val_f1score: 0.0000e+00
Epoch 3/10
28698/28698 [==============================] - 3058s 107ms/sample - loss: 1.8121 - acc: 0.2514 - f1score: 0.0000e+00 - val_loss: 1.8155 - val_acc: 0.2449 - val_f1score: 0.0000e+00
Epoch 4/10
28698/28698 [==============================] - 3042s 106ms/sample - loss: 1.8118 - acc: 0.2514 - f1score: 0.0000e+00 - val_loss: 1.8170 - val_acc: 0.2449 - val_f1score: 0.0000e+00
Epoch 5/10
28698/28698 [==============================] - 3062s 107ms/sample - loss: 1.8118 - acc: 0.2514 - f1score: 0.0000e+00 - val_loss: 1.8160 - val_acc: 0.2449 - val_f1score: 0.0000e

Too many parameters compared to the data. <br>
So it doesn't work.

### 2) My Alexnet

In [0]:
# 2. my alexnet
# Alexnet을 개조
# Data Augmentation은 컴퓨터 성능의 한계로 못하기 때문에 변형함.

# 주의 !!!!기본은 모형을 조절하는 것이 아닌, 데이터를 뻥튀기 하는 것임을 늘 잊지말기!!!!

# data import
x_train = pd.read_csv("mydata/X_train.csv",header=0,index_col=0)
x_valid = pd.read_csv("mydata/X_private_test.csv",header=0,index_col=0)
x_test = pd.read_csv("mydata/X_public_test.csv",header=0,index_col=0)
y_train = pd.read_csv("mydata/y_train.csv",header=0,index_col=0)
y_valid = pd.read_csv("mydata/y_private_test.csv",header=0,index_col=0)
y_test = pd.read_csv("mydata/y_public_test.csv",header=0,index_col=0)

In [0]:
# data handling
size = 57 # 적당한 크기로 잡음.
x_train = np.array(x_train).reshape([-1,48,48,1])

In [14]:
x_train.shape

(28698, 48, 48, 1)

In [0]:
x_train_zoom = np.zeros([x_train.shape[0],size,size,1],dtype="float32")
for i in range(x_train.shape[0]):
    x_train_zoom[i,:] = cv2.resize(x_train[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,1) /255

In [16]:
x_train_zoom.shape

(28698, 57, 57, 1)

In [0]:
x_valid = np.array(x_valid).reshape([-1,48,48,1])
x_valid_zoom = np.zeros([x_valid.shape[0],size,size,1],dtype="float32")
for i in range(x_valid.shape[0]):
    x_valid_zoom[i,:] = cv2.resize(x_valid[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,1) /255

In [18]:
x_valid_zoom.shape

(3589, 57, 57, 1)

In [0]:
x_test = np.array(x_test).reshape([-1,48,48,1])
x_test_zoom = np.zeros([x_test.shape[0],size,size,1],dtype="float32")
for i in range(x_test.shape[0]):
    x_test_zoom[i,:] = cv2.resize(x_test[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,1) /255

In [20]:
x_test_zoom.shape

(3588, 57, 57, 1)

In [0]:
# 다음의 절차로 모형을 개조한다.

# 1. 227의 1/4 연산인 57로 이미지사이즈를 재조정한다.
# 2. 모수와 관련이 가장 깊은 fc층에서, 기존의 4096개의 노드를 1/16 (비율) 배 만큼, 즉 256개로줄인다.
# 3. 다음과 같이 모형을 재구성한다.

# convolution layer
# 입력 : 57x57x1
# 첫번째 층 : 5x5 필터 24장, strides = 2 -> maxpooling 3x3 , stirdes = 2   ===> 13 x 13 x 24 feature map
# 두번째 층 : 3x3 필터 64장, strides = 1, padding = "same"                 ===> 13 x 13 x 64 feature map
# 세번째 층 : 3x3 필터 96장, strides = 1                                   ===> 11 x 11 x 96 feature map
# 네번째 층 : 3x3 필터 96장, strides = 1, padding = "same"                 ===> 11 x 11 x 96 feature map
# 다섯째 층 : 3x3 필터 64장, strides = 1 -> maxpooling 3x3 , strides = 2   ===> 4 x 4 x 64 feature map

# fc layer
# 여섯째 층 : 노드 256개, dropout = 0.5
# 일곱째 층 : 노드 256개, dropout = 0.5
# 여덟째 층 : 노드 7개



def Alexnet(img_shape=(57, 57, 1), n_classes=10, l2_reg=0.,
	weights=None):

	# Initialize model
	alexnet = Sequential()

	# Layer 1
	alexnet.add(Conv2D(24, (5, 5), input_shape=img_shape, strides=2,kernel_regularizer=l2(l2_reg)))
	alexnet.add(Activation('relu'))
	alexnet.add(LRN(name='layer1_LRN'))
	alexnet.add(MaxPooling2D(pool_size=(3, 3),strides=2))

	# Layer 2
	alexnet.add(Conv2D(64, (3, 3), padding='same',strides=1,kernel_regularizer=l2(l2_reg)))
	alexnet.add(Activation('relu'))
	alexnet.add(LRN(name='layer2_LRN'))

	# Layer 3
	alexnet.add(Conv2D(96, (3, 3),  strides=1,kernel_regularizer=l2(l2_reg)))
	alexnet.add(Activation('relu'))
	alexnet.add(LRN(name='layer3_LRN'))

	# Layer 4
	alexnet.add(Conv2D(96, (3, 3), padding='same'))
	alexnet.add(Activation('relu'))
	alexnet.add(LRN(name='layer4_LRN'))

	# Layer 5
	alexnet.add(Conv2D(64, (3, 3)))
	alexnet.add(Activation('relu'))
	alexnet.add(LRN(name='layer5_LRN'))
	alexnet.add(MaxPooling2D(pool_size=(3, 3),strides=2))

	# Layer 6
	alexnet.add(Flatten())
	alexnet.add(Dense(256,kernel_regularizer=l2(l2_reg)))

	alexnet.add(Activation('relu'))
	alexnet.add(Dropout(0.5))

	# Layer 7
	alexnet.add(Dense(256,kernel_regularizer=l2(l2_reg)))
	alexnet.add(Activation('relu'))
	alexnet.add(Dropout(0.5))

	# Layer 8
	alexnet.add(Dense(n_classes))
	alexnet.add(Activation('softmax'))

	if weights is not None:
		alexnet.load_weights(weights)

	return alexnet

In [0]:
#내 데이터 맞춤형 모형
model = Alexnet(img_shape=(57, 57, 1), n_classes=7, l2_reg=0.,weights=None)

In [22]:
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 27, 27, 24)        624       
_________________________________________________________________
activation (Activation)      (None, 27, 27, 24)        0         
_________________________________________________________________
layer1_LRN (LRN)             (None, 27, 27, 24)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 24)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 64)        13888     
_________________________________________________________________
activation_1 (Activation)    (None, 13, 13, 64)        0         
_________________________________________

In [0]:
# 여기는 학습의 효율을 위해. Adam으로 넘어간다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='sparse_categorical_crossentropy',
              metrics=['accuracy',f1score])

In [25]:
model.fit(x_train_zoom,y_train,batch_size=128, validation_data=(x_valid_zoom,y_valid) , epochs=30)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 28698 samples, validate on 3589 samples
Epoch 1/30
28698/28698 [==============================] - 162s 6ms/sample - loss: 1.8253 - acc: 0.2447 - f1score: 0.0000e+00 - val_loss: 1.8173 - val_acc: 0.2449 - val_f1score: 0.0000e+00
Epoch 2/30
28698/28698 [==============================] - 160s 6ms/sample - loss: 1.7211 - acc: 0.3036 - f1score: 0.1744 - val_loss: 1.5881 - val_acc: 0.3798 - val_f1score: 0.3118
Epoch 3/30
28698/28698 [==============================] - 161s 6ms/sample - loss: 1.5451 - acc: 0.3936 - f1score: 0.3714 - val_loss: 1.4838 - val_acc: 0.4235 - val_f1score: 0.4127
Epoch 4/30
28698/28698 [==============================] - 161s 6ms/sample - loss: 1.4434 - acc: 0.4368 - f1score: 0.4403 - val_loss: 1.3840 - val_acc: 0.4567 - val_f1score: 0.4974
Epoch 5/30
28698/28698 [==============================] - 165s 6ms/sample - loss: 1.3770 - acc: 0.4662 - f1score: 0.4840 - val_lo

Instead of touching the original model, we need to proceed with data augmentation. <br>
However, because of the limitation of computer performance, the code is changed like this.

My Alexnet <br>
In Epoch 10/30
<br>
Train // $\quad$ Accuracy : 0.5622 $\quad$ F1 Score : 0.6406  <br>
Valid //  $\quad$ Accuracy : 0.5506 $\quad$  F1 Score : 0.6850 

In [1]:
# _, acc, f1 = model.evaluate(x_test_zoom,y_test,batch_size=128)  # early stopping으로 training에서  epoch 10번후 적용해야함.
# print("\nAccuracy: {:.4f}, F1 Score: {:.4f}".format(acc,f1))